In [ ]:
import ee
ee.Initialize()
from IPython.display import display, Image

In [ ]:
scale = 20

In [ ]:
roi = ee.FeatureCollection("users/nkeikon/myanmar_sr/area1")
op = ee.FeatureCollection("users/nkeikon/myanmar_sr/OP_young")
PA = ee.FeatureCollection("WCMC/WDPA/current/polygons")
s2TNI = ee.Image("users/nkeikon/myanmar_sr/s2image").clip(roi)
palm = ee.FeatureCollection("users/nkeikon/myanmar_sr/palm_area1")
rubber = ee.FeatureCollection("users/nkeikon/myanmar_sr/rubber_area1")
other = ee.FeatureCollection("users/nkeikon/myanmar_sr/other_area1")
bare = ee.FeatureCollection("users/nkeikon/myanmar_sr/bare_area1")
water = ee.FeatureCollection("users/nkeikon/myanmar_sr/water_area1")
shrub = ee.FeatureCollection("users/nkeikon/myanmar_sr/shrub_area1")

In [ ]:
Image(
    url=s2TNI.getThumbUrl(
        {"bands": "red_mean,green_mean,blue_mean", "min": 0, "max": 0.4, "size": "400"}
    )
)

In [ ]:
# Add Sentinel-1 data (only ascending for Jan to Mar 2018 due to noise)
s1_JanMar18 = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filter(ee.Filter.eq("instrumentMode", "IW"))
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VH"))
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV"))
    .filter(ee.Filter.eq("orbitProperties_pass", "ASCENDING"))
    .filterDate("2018-01-01", "2018-03-31")
    .filterBounds(roi)
)

s1_Apr18Jan19 = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filter(ee.Filter.eq("instrumentMode", "IW"))
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VH"))
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV"))
    .filterDate("2018-04-01", "2019-01-31")
    .filterBounds(roi)
)

s1 = s1_JanMar18.merge(s1_Apr18Jan19)

s1mean = s1.mean().select(["VV", "VH"], ["VV_mean", "VH_mean"]).clip(roi)
s1stdev = s1.reduce(ee.Reducer.stdDev()).clip(roi)

In [ ]:
# Compute the Normalized Difference Vegetation Index (NDVI)
red = s2TNI.select("red_mean")
nir = s2TNI.select("nir_mean")
ndvi = nir.subtract(red).divide(nir.add(red)).rename("NDVI")

# Compute standard deviation (stDev) as texture of the NDVI
texture = ndvi.reduceNeighborhood(
    reducer=ee.Reducer.stdDev(), kernel=ee.Kernel.square(5)
)

# Compute elevation, slope and aspect
SRTM = ee.Image("USGS/SRTMGL1_003").clip(roi)
slope = ee.Terrain.slope(SRTM)

# Add them to bands
s2final = (
    ee.Image(s2TNI)
    .addBands(ndvi)
    .addBands(texture)
    .addBands(slope)
    .addBands(s1mean)
    .addBands(s1stdev)
    .float()
)

In [ ]:
bands = [
    "blue_mean",
    "green_mean",
    "red_mean",
    "red1_mean",
    "red2_mean",
    "red3_mean",
    "nir_mean",
    "red4_mean",
    "swir1_mean",
    "swir2_mean",
    "NDVI",
    "NDVI_stdDev",
    "slope",
    "VH_mean",
    "VV_mean",
    "VH_stdDev",
    "VV_stdDev",
]
s2finalWbands = s2final.select(bands)

# Scale the image for classification
s2classification = s2finalWbands.reproject(
    ee.Projection("EPSG:4326").atScale(scale)
).reduceResolution(reducer=ee.Reducer.mean(), maxPixels=65536)

In [ ]:
randomSeed = 0
n = randomSeed
split = 0.5

# Get the values for all pixels in each polygon in the training.
palmSample = s2classification.sampleRegions(
    collection=palm, properties=["class"], scale=scale
)
rubberSample = s2classification.sampleRegions(
    collection=rubber, properties=["class"], scale=scale
)
otherSample = s2classification.sampleRegions(
    collection=other, properties=["class"], scale=scale
)
shrubSample = s2classification.sampleRegions(
    collection=shrub, properties=["class"], scale=scale
)
bareSample = s2classification.sampleRegions(
    collection=bare, properties=["class"], scale=scale
)
waterSample = s2classification.sampleRegions(
    collection=water, properties=["class"], scale=scale
)

randomPalm = palmSample.randomColumn("random", n)
randomRubber = rubberSample.randomColumn("random", n)
randomOther = otherSample.randomColumn("random", n)
randomShrub = shrubSample.randomColumn("random", n)
randomBare = bareSample.randomColumn("random", n)
randomWater = waterSample.randomColumn("random", n)

trainingSample = (
    randomPalm.filter(ee.Filter.lt("random", split))
    .merge(randomRubber.filter(ee.Filter.lt("random", split)))
    .merge(randomOther.filter(ee.Filter.lt("random", split)))
    .merge(randomShrub.filter(ee.Filter.lt("random", split)))
    .merge(randomBare.filter(ee.Filter.lt("random", split)))
    .merge(randomWater.filter(ee.Filter.lt("random", split)))
)

testingSample = (
    randomPalm.filter(ee.Filter.gte("random", split))
    .merge(randomRubber.filter(ee.Filter.gte("random", split)))
    .merge(randomOther.filter(ee.Filter.gte("random", split)))
    .merge(randomShrub.filter(ee.Filter.gte("random", split)))
    .merge(randomBare.filter(ee.Filter.gte("random", split)))
    .merge(randomWater.filter(ee.Filter.gte("random", split)))
)

In [ ]:
print("palm", randomPalm.size().getInfo())
print("rubber", randomRubber.size().getInfo())
print("other", randomOther.size().getInfo())
print("shrub", randomShrub.size().getInfo())
print("bare", randomBare.size().getInfo())
print("water", randomWater.size().getInfo())
# print('training', trainingSample.size().getInfo())
# print('testing', testingSample.size().getInfo())

In [ ]:
classifier = ee.Classifier.randomForest(numberOfTrees=100, variablesPerSplit=4).train(
    trainingSample, "class", bands
)

classified = s2classification.classify(classifier)

In [ ]:
# Get a confusion matrix representing resubstitution accuracy
# trainAccuracy = classifier.confusionMatrix()
# print('Resubstitution error matrix: ', trainAccuracy.getInfo())
# print('Training overall accuracy: ', trainAccuracy.accuracy().getInfo())

# Classify the validation data
validated = testingSample.classify(classifier)

# Get a confusion matrix representing expected accuracy
testAccuracy = validated.errorMatrix("class", "classification",[1,2,3,4,5,6])
errorMatrix = testAccuracy.array()

overallAccuracy = testAccuracy.accuracy()
producersAccuracy = testAccuracy.producersAccuracy()
consumersAccuracy = testAccuracy.consumersAccuracy()

feature = ee.Feature(None)
feature = feature.set("matrix", errorMatrix)
feature = feature.set("results", overallAccuracy)
feature = feature.set("producer", producersAccuracy)
feature = feature.set("consumer", consumersAccuracy)
accuracy_results = ee.FeatureCollection(feature)

# Export as csv
export1 = ee.batch.Export.table.toDrive(
    collection=accuracy_results,
    description="export_area1_accuracy",
    fileNamePrefix="area1_accuracy",
)
export1.start()

In [ ]:
# Run a majority filter
# radius (Float, default: 1.5) kernelType (String, default: "circle"):units (String, default: "pixels")
filtered = classified.focal_mode()

In [ ]:
validation = filtered.sampleRegions(
    collection=testingSample, properties=["class"], scale=scale
)

# Classify the validation data
validated = validation.classify(classifier)

# Get a confusion matrix representing expected accuracy
testAccuracy = validation.errorMatrix("class", "classification",[1,2,3,4,5,6])
errorMatrix = testAccuracy.array()

overallAccuracy = testAccuracy.accuracy()
producersAccuracy = testAccuracy.producersAccuracy()
consumersAccuracy = testAccuracy.consumersAccuracy()

feature = ee.Feature(None)
feature = feature.set("matrix", errorMatrix)
feature = feature.set("results", overallAccuracy)
feature = feature.set("producer", producersAccuracy)
feature = feature.set("consumer", consumersAccuracy)
accuracy_results = ee.FeatureCollection(feature)

# Export as csv
export2 = ee.batch.Export.table.toDrive(
    collection=accuracy_results,
    description="export_area1_accuracy",
    fileNamePrefix="area1_accuracy",
)
export2.start()

In [ ]:
# Export the image to estimate area
# to drive
export3 = ee.batch.Export.image.toDrive(
    image=filtered,
    description="export_area1_classified_drive",
    fileNamePrefix="area1_classified",
    scale=scale,
    maxPixels=1e13,
)
export3.start()

# to asset (replace with your username)
export4 = ee.batch.Export.image.toAsset(
    image=filtered,
    description="export_area1_classified_asset",
    assetId="users/nkeikon/area1_classified",
    scale=20,
    maxPixels=1e13,
)
# export4.start()

In [ ]:
# Produce final map by manually classifying one young oil palm plantation
area1_finalmap = filtered.where(
    (classified.clip(op).eq(2).Or(classified.clip(op).eq(3))), 1
)

export5 = ee.batch.Export.image.toDrive(
    image=area1_finalmap,
    description="export_area1_map_drive",
    fileNamePrefix="area1_map",
    scale=scale,
    maxPixels=1e13,
)
export5.start()

# to asset (replace with your username)
export6 = ee.batch.Export.image.toAsset(
    image=area1_finalmap,
    description="export_area1_map_asset",
    assetId="users/nkeikon/area1_map",
    scale=20,
    maxPixels=1e13,
)
# export6.start()

In [ ]:
# query current tasks
tasks = ee.batch.Task.list()
print(tasks[0])
print(tasks[1])
print(tasks[2])
print(tasks[3])
print(tasks[4])
print(tasks[5])